# MCP Leaderboard Dashboard

Track adoption metrics for Payment, Commerce, and Crypto MCP (Model Context Protocol) servers in real-time.

**Data Sources:**
- NPM Registry API: Weekly downloads and trends
- GitHub REST API: Stars, forks, issues, last commit

**Note:** This notebook includes time delays between API calls to avoid rate limiting.

In [ ]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import time

# Rate limiting configuration
API_DELAY_SECONDS = 1.0  # Delay between API calls to avoid rate limits

def rate_limited_request(url, headers=None, delay=API_DELAY_SECONDS):
    """Make a rate-limited HTTP request with delay."""
    time.sleep(delay)
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

In [ ]:
# MCP Registry - All tracked MCP servers
mcp_registry = [
    # Payments
    {"id": "stripe-mcp", "name": "Stripe MCP", "category": "payments", "npm_package": "@stripe/mcp", "github_repo": "stripe/ai", "description": "Official Stripe MCP server"},
    {"id": "stripe-agent-toolkit", "name": "Stripe Agent Toolkit", "category": "payments", "npm_package": "@stripe/agent-toolkit", "github_repo": "stripe/ai", "description": "LangChain/Vercel AI SDK integration"},
    {"id": "paypal-mcp", "name": "PayPal MCP", "category": "payments", "npm_package": "@paypal/mcp", "github_repo": "paypal/agent-toolkit", "description": "PayPal agent toolkit"},
    {"id": "square-mcp", "name": "Square MCP", "category": "payments", "npm_package": None, "github_repo": "square/square-mcp-server", "description": "Square payments MCP"},
    
    # Commerce
    {"id": "shopify-dev-mcp", "name": "Shopify Dev MCP", "category": "commerce", "npm_package": "@shopify/dev-mcp", "github_repo": None, "description": "Official Shopify developer MCP"},
    {"id": "shopify-mcp", "name": "Shopify MCP", "category": "commerce", "npm_package": "shopify-mcp", "github_repo": "GeLi2001/shopify-mcp", "description": "Community Shopify MCP"},
    {"id": "shopify-storefront-mcp", "name": "Shopify Storefront MCP", "category": "commerce", "npm_package": "@wolfielabs/shopify-storefront-mcp-server", "github_repo": None, "description": "Shopify storefront integration"},
    {"id": "shopify-mcp-server", "name": "Shopify MCP Server", "category": "commerce", "npm_package": "shopify-mcp-server", "github_repo": None, "description": "Community Shopify server"},
    
    # Crypto
    {"id": "x402-mcp", "name": "x402 MCP", "category": "crypto", "npm_package": "x402-mcp", "github_repo": "ethanniser/x402-mcp", "description": "Vercel x402 protocol integration"},
    {"id": "mcpay", "name": "MCPay", "category": "crypto", "npm_package": "mcpay", "github_repo": "microchipgnu/MCPay", "description": "x402 payment infrastructure"},
    {"id": "coingecko-mcp", "name": "CoinGecko MCP", "category": "crypto", "npm_package": "coingecko-mcp", "github_repo": None, "description": "Crypto price data"},
    {"id": "hive-crypto-mcp", "name": "Hive Intelligence", "category": "crypto", "npm_package": None, "github_repo": "hive-intel/hive-crypto-mcp", "description": "DeFi analytics"},
    {"id": "near-mcp", "name": "NEAR MCP", "category": "crypto", "npm_package": "@nearai/near-mcp", "github_repo": "nearai/near-mcp", "description": "NEAR blockchain integration"},
    {"id": "coinbase-cdp", "name": "Coinbase CDP", "category": "crypto", "npm_package": "@coinbase/agentkit-model-context-protocol", "github_repo": "coinbase/agentkit", "description": "Coinbase Developer Platform MCP"},
]

print(f"Tracking {len(mcp_registry)} MCP servers across 3 categories")

In [ ]:
# NPM API Functions with rate limiting

def get_weekly_downloads(package_name):
    """Fetch weekly downloads for an npm package with rate limiting."""
    if not package_name:
        return 0
    
    encoded_name = requests.utils.quote(package_name, safe='')
    url = f"https://api.npmjs.org/downloads/point/last-week/{encoded_name}"
    
    data = rate_limited_request(url)
    if data and 'downloads' in data:
        return data['downloads']
    return 0


def get_downloads_trend(package_name, days=7):
    """Fetch daily downloads for trend visualization with rate limiting."""
    if not package_name:
        return []
    
    encoded_name = requests.utils.quote(package_name, safe='')
    url = f"https://api.npmjs.org/downloads/range/last-month/{encoded_name}"
    
    data = rate_limited_request(url)
    if data and 'downloads' in data:
        # Get last N days of downloads
        downloads = [d['downloads'] for d in data['downloads'][-days:]]
        return downloads
    return []


def get_downloads_change_percent(package_name):
    """Calculate week-over-week download change percentage with rate limiting."""
    if not package_name:
        return 0
    
    encoded_name = requests.utils.quote(package_name, safe='')
    url = f"https://api.npmjs.org/downloads/range/last-month/{encoded_name}"
    
    data = rate_limited_request(url)
    if data and 'downloads' in data and len(data['downloads']) >= 14:
        downloads = data['downloads']
        current_week = sum(d['downloads'] for d in downloads[-7:])
        previous_week = sum(d['downloads'] for d in downloads[-14:-7])
        
        if previous_week > 0:
            return round(((current_week - previous_week) / previous_week) * 100, 1)
    return 0

print("NPM API functions loaded with rate limiting")

In [ ]:
# GitHub API Functions with rate limiting

# Optional: Set your GitHub token for higher rate limits (60/hr -> 5000/hr)
GITHUB_TOKEN = None  # Set to your token string if available

def get_github_metrics(repo_path):
    """Fetch GitHub repository metrics with rate limiting."""
    if not repo_path:
        return None
    
    url = f"https://api.github.com/repos/{repo_path}"
    headers = {'Accept': 'application/vnd.github.v3+json'}
    
    if GITHUB_TOKEN:
        headers['Authorization'] = f'Bearer {GITHUB_TOKEN}'
    
    data = rate_limited_request(url, headers=headers)
    if data:
        return {
            'stars': data.get('stargazers_count', 0),
            'forks': data.get('forks_count', 0),
            'open_issues': data.get('open_issues_count', 0),
            'last_commit': data.get('pushed_at')
        }
    return None

print("GitHub API functions loaded with rate limiting")

In [ ]:
# Fetch all MCP data with progress tracking

print(f"Fetching data for {len(mcp_registry)} MCPs...")
print(f"Using {API_DELAY_SECONDS}s delay between API calls to avoid rate limits.")
print("This may take a few minutes.\n")

mcp_data = []

for i, mcp in enumerate(mcp_registry):
    print(f"[{i+1}/{len(mcp_registry)}] Fetching {mcp['name']}...")
    
    # Fetch NPM metrics
    npm_downloads = get_weekly_downloads(mcp['npm_package'])
    npm_change = get_downloads_change_percent(mcp['npm_package'])
    npm_trend = get_downloads_trend(mcp['npm_package'])
    
    # Fetch GitHub metrics
    github = get_github_metrics(mcp['github_repo'])
    
    mcp_data.append({
        'id': mcp['id'],
        'name': mcp['name'],
        'category': mcp['category'],
        'description': mcp['description'],
        'npm_package': mcp['npm_package'],
        'github_repo': mcp['github_repo'],
        'npm_downloads_weekly': npm_downloads,
        'npm_change_percent': npm_change,
        'npm_trend': npm_trend,
        'github_stars': github['stars'] if github else 0,
        'github_forks': github['forks'] if github else 0,
        'open_issues': github['open_issues'] if github else 0,
        'last_commit': github['last_commit'] if github else None
    })

print("\nData fetching complete!")

In [ ]:
# Create main DataFrame and calculate rankings

df_mcps = pd.DataFrame(mcp_data)

# Sort by weekly downloads (primary) and stars (secondary)
df_mcps = df_mcps.sort_values(
    by=['npm_downloads_weekly', 'github_stars'], 
    ascending=[False, False]
).reset_index(drop=True)

# Add rank column
df_mcps['rank'] = df_mcps.index + 1

# Format last commit as relative time
def format_last_commit(commit_time):
    if not commit_time:
        return 'N/A'
    try:
        dt = datetime.fromisoformat(commit_time.replace('Z', '+00:00'))
        days_ago = (datetime.now(dt.tzinfo) - dt).days
        if days_ago == 0:
            return 'Today'
        elif days_ago == 1:
            return '1 day ago'
        else:
            return f'{days_ago} days ago'
    except:
        return 'N/A'

df_mcps['last_commit_display'] = df_mcps['last_commit'].apply(format_last_commit)

print(f"Processed {len(df_mcps)} MCPs")
print(f"Total weekly downloads: {df_mcps['npm_downloads_weekly'].sum():,}")

## Summary Metrics

In [ ]:
# Calculate summary metrics by category

total_mcps = len(df_mcps)
total_downloads = df_mcps['npm_downloads_weekly'].sum()
total_stars = df_mcps['github_stars'].sum()

category_stats = df_mcps.groupby('category').agg({
    'id': 'count',
    'npm_downloads_weekly': 'sum',
    'github_stars': 'sum'
}).rename(columns={'id': 'count'})

print("="*50)
print("MCP LEADERBOARD SUMMARY")
print("="*50)
print(f"\nTotal MCPs Tracked: {total_mcps}")
print(f"Total Weekly Downloads: {total_downloads:,}")
print(f"Total GitHub Stars: {total_stars:,}")
print("\nBy Category:")
print("-"*50)
for cat in ['payments', 'commerce', 'crypto']:
    if cat in category_stats.index:
        row = category_stats.loc[cat]
        print(f"{cat.upper():12} | {int(row['count']):2} MCPs | {int(row['npm_downloads_weekly']):,} downloads | {int(row['github_stars']):,} stars")

## Leaderboard Table

In [ ]:
# Display the main leaderboard

leaderboard_cols = ['rank', 'name', 'category', 'npm_downloads_weekly', 'npm_change_percent', 'github_stars', 'github_forks', 'last_commit_display']
df_leaderboard = df_mcps[leaderboard_cols].copy()
df_leaderboard.columns = ['Rank', 'Name', 'Category', 'Weekly Downloads', 'Change %', 'Stars', 'Forks', 'Last Commit']

# Format numbers
df_leaderboard['Weekly Downloads'] = df_leaderboard['Weekly Downloads'].apply(lambda x: f"{x:,}" if x > 0 else '-')
df_leaderboard['Change %'] = df_leaderboard['Change %'].apply(lambda x: f"+{x}%" if x > 0 else f"{x}%" if x < 0 else '-')
df_leaderboard['Stars'] = df_leaderboard['Stars'].apply(lambda x: f"{x:,}" if x > 0 else '-')
df_leaderboard['Forks'] = df_leaderboard['Forks'].apply(lambda x: f"{x:,}" if x > 0 else '-')
df_leaderboard['Category'] = df_leaderboard['Category'].str.capitalize()

df_leaderboard

## Weekly Downloads by MCP

In [ ]:
# Bar chart: Weekly downloads by MCP

df_chart = df_mcps[df_mcps['npm_downloads_weekly'] > 0].copy()
df_chart = df_chart.sort_values('npm_downloads_weekly', ascending=True)

category_colors = {
    'payments': '#6366f1',  # Indigo
    'commerce': '#22c55e',  # Green
    'crypto': '#f59e0b'     # Amber
}

fig = px.bar(
    df_chart,
    y='name',
    x='npm_downloads_weekly',
    color='category',
    orientation='h',
    title='Weekly NPM Downloads by MCP',
    labels={'npm_downloads_weekly': 'Weekly Downloads', 'name': '', 'category': 'Category'},
    color_discrete_map=category_colors
)

fig.update_layout(
    height=500,
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

## Downloads by Category

In [ ]:
# Pie chart: Downloads distribution by category

df_cat = df_mcps.groupby('category')['npm_downloads_weekly'].sum().reset_index()
df_cat.columns = ['Category', 'Downloads']
df_cat['Category'] = df_cat['Category'].str.capitalize()

fig = px.pie(
    df_cat,
    values='Downloads',
    names='Category',
    title='Weekly Downloads by Category',
    color='Category',
    color_discrete_map={'Payments': '#6366f1', 'Commerce': '#22c55e', 'Crypto': '#f59e0b'}
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=400)

fig.show()

## GitHub Stars vs Downloads

In [ ]:
# Scatter plot: GitHub stars vs npm downloads

df_scatter = df_mcps[(df_mcps['npm_downloads_weekly'] > 0) | (df_mcps['github_stars'] > 0)].copy()

fig = px.scatter(
    df_scatter,
    x='npm_downloads_weekly',
    y='github_stars',
    color='category',
    size='github_forks',
    size_max=40,
    hover_name='name',
    title='GitHub Stars vs NPM Downloads',
    labels={
        'npm_downloads_weekly': 'Weekly Downloads',
        'github_stars': 'GitHub Stars',
        'category': 'Category'
    },
    color_discrete_map=category_colors
)

fig.update_layout(height=500)
fig.show()

## 7-Day Download Trends

In [ ]:
# Line chart: 7-day download trends for top MCPs

# Get top 6 MCPs by downloads that have trend data
top_mcps = df_mcps[df_mcps['npm_trend'].apply(len) > 0].head(6)

fig = go.Figure()

days = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7']

for _, mcp in top_mcps.iterrows():
    trend = mcp['npm_trend']
    if len(trend) == 7:
        color = category_colors.get(mcp['category'], '#888')
        fig.add_trace(go.Scatter(
            x=days,
            y=trend,
            mode='lines+markers',
            name=mcp['name'],
            line=dict(width=2),
            marker=dict(size=6)
        ))

fig.update_layout(
    title='7-Day Download Trends (Top MCPs)',
    xaxis_title='Day',
    yaxis_title='Daily Downloads',
    height=450,
    hovermode='x unified'
)

fig.show()

## Week-over-Week Change

In [ ]:
# Bar chart: Week-over-week change percentage

df_change = df_mcps[df_mcps['npm_change_percent'] != 0].copy()
df_change = df_change.sort_values('npm_change_percent', ascending=True)

# Color based on positive/negative change
df_change['color'] = df_change['npm_change_percent'].apply(
    lambda x: '#22c55e' if x > 0 else '#ef4444'
)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=df_change['name'],
    x=df_change['npm_change_percent'],
    orientation='h',
    marker_color=df_change['color'],
    text=df_change['npm_change_percent'].apply(lambda x: f"+{x}%" if x > 0 else f"{x}%"),
    textposition='outside'
))

fig.update_layout(
    title='Week-over-Week Download Change',
    xaxis_title='Change %',
    yaxis_title='',
    height=450,
    showlegend=False
)

# Add vertical line at 0
fig.add_vline(x=0, line_dash='dash', line_color='gray')

fig.show()

## Category Breakdown Tables

In [ ]:
# Payments MCPs
print("PAYMENTS MCPs")
print("="*60)
df_payments = df_mcps[df_mcps['category'] == 'payments'][['rank', 'name', 'npm_downloads_weekly', 'github_stars', 'description']].copy()
df_payments.columns = ['Rank', 'Name', 'Downloads', 'Stars', 'Description']
df_payments

In [ ]:
# Commerce MCPs
print("COMMERCE MCPs")
print("="*60)
df_commerce = df_mcps[df_mcps['category'] == 'commerce'][['rank', 'name', 'npm_downloads_weekly', 'github_stars', 'description']].copy()
df_commerce.columns = ['Rank', 'Name', 'Downloads', 'Stars', 'Description']
df_commerce

In [ ]:
# Crypto MCPs
print("CRYPTO MCPs")
print("="*60)
df_crypto = df_mcps[df_mcps['category'] == 'crypto'][['rank', 'name', 'npm_downloads_weekly', 'github_stars', 'description']].copy()
df_crypto.columns = ['Rank', 'Name', 'Downloads', 'Stars', 'Description']
df_crypto

## Export Data

In [ ]:
# Export full data (optional - uncomment to use)

# Export columns
export_cols = ['rank', 'name', 'category', 'npm_package', 'github_repo', 'npm_downloads_weekly', 
               'npm_change_percent', 'github_stars', 'github_forks', 'open_issues', 'last_commit', 'description']

df_export = df_mcps[export_cols].copy()
df_export['fetched_at'] = datetime.now().isoformat()

# Uncomment to export:
# df_export.to_csv('mcp_leaderboard_data.csv', index=False)
# print("Data exported to mcp_leaderboard_data.csv")

print("Export DataFrame ready. Uncomment the lines above to save to CSV.")
df_export.head()

---

## Hex Input Parameters

To add interactivity in Hex, create these input parameters:

| Parameter Name | Type | Description | Default Value |
|---------------|------|-------------|---------------|
| `selected_category` | Dropdown | Filter by category | "all" |
| `min_downloads` | Slider | Minimum weekly downloads | 0 |

Then modify the filtering cells to use these parameters as variables.

---

**Data Sources:**
- [NPM Registry API](https://github.com/npm/registry/blob/master/docs/download-counts.md) - Rate limit: 1000/min
- [GitHub REST API](https://docs.github.com/en/rest) - Rate limit: 60/hr (unauth), 5000/hr (with token)

**Rate Limiting:** This notebook includes configurable delays between API calls (`API_DELAY_SECONDS`) to avoid hitting rate limits.